In [1]:
import numpy as np
import tensorflow as tf
import keras
import cv2
from keras.layers import MaxPool2D,Conv2D,UpSampling2D,Input,Dropout
from keras.models import Sequential
from keras.utils import img_to_array, plot_model
from keras.utils.vis_utils import plot_model
import os
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
import gc

In [2]:
def sorted_alphanumeric(data):  
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)',key)]
    return sorted(data,key = alphanum_key)
# defining the size of the image
SIZE = 256
color_img = []
gray_img = []
path = 'E:/TER/landscape Images/color'
files = os.listdir(path)
files = sorted_alphanumeric(files)
for i in tqdm(files):    
        if i == '7200.jpg':
            break
        else:    
            img = cv2.imread(path + '/'+i,1)
            img = cv2.resize(img, (SIZE, SIZE))
            # open cv reads images in BGR format so we have to convert it to LAB
            img = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)
            #resizing image
            #分离图像通道
            l, a, b = cv2.split(img)
            xx=b
            #归一化
            l = (l/ 255.0)
            a = (a/ 255.0)
            b = (b/ 255.0)
            #合并通道
            img = cv2.merge([l, a, b])
            color_img.append(img_to_array(img))
            gray_img.append(img_to_array(l))
gc.collect()

100%|██████████| 7129/7129 [00:11<00:00, 629.83it/s]


9

In [3]:
test_gray_image = gray_img
test_color_image = color_img
del gray_img
del color_img
gc.collect()
# reshaping
test_gray_image = np.reshape(test_gray_image,(len(test_gray_image),SIZE,SIZE,1))
test_color_image = np.reshape(test_color_image, (len(test_color_image),SIZE,SIZE,3))
print('Test color image shape',test_color_image.shape)

Test color image shape (7129, 256, 256, 3)


In [4]:
autoencoder_lab = keras.models.load_model('E:/TER/model/autoencoder_lab.h5')

In [5]:
def plot_images(color,grayscale,predicted,n):
    M = tf.keras.losses.MeanSquaredError()
    MSE = M(color, predicted)
    plt.figure(figsize=(15,5))
    l, a, b = cv2.split(np.array(color))
    #归一化
    l = (l.astype('float32')* 100)
    a = (a.astype('float32')* 255.0)-128
    b = (b.astype('float32')* 255.0)-128
    #合并通道
    target_image = cv2.merge([l, a, b])  
    l, a, b = cv2.split(np.array(predicted))
    #归一化
    l = (l.astype('float32')* 100)
    a = (a.astype('float32')* 255.0)-128
    b = (b.astype('float32')* 255.0)-128
    #合并通道
    prediction_image = cv2.merge([l, a, b])
    target_image = cv2.cvtColor(target_image, cv2.COLOR_LAB2RGB)
    prediction_image = cv2.cvtColor(prediction_image, cv2.COLOR_LAB2RGB)
    display_list = [grayscale, target_image, prediction_image]
    title = ['Input Image', 'Ground Truth', 'Predicted Image in LAB']
    for i in range(3):
        plt.subplot(1, 3, i+1)
        plt.title(title[i])
        plt.imshow(display_list[i],cmap='gray')
        plt.axis('off')
    plt.suptitle("MSE: "+str(MSE.numpy()), fontsize = 20)
    plt.imsave('E:/TER/Result_autoencoder_landscape_LAB/output/'+str(n)+'.png',prediction_image)
    plt.savefig('E:/TER/Result_autoencoder_landscape_LAB/plot/'+str(n)+'.png')
    plt.close()
predicted = np.zeros((7129,SIZE, SIZE,3))
predicted[:1782] = np.clip(autoencoder_lab.predict(test_gray_image[:1782].reshape(1782,SIZE, SIZE,1)),0.0,1.0).reshape(1782,SIZE, SIZE,3)
predicted[1782:3564] = (np.clip(autoencoder_lab.predict(test_gray_image[1782:3564].reshape(1782,SIZE, SIZE,1)),0.0,1.0).reshape(1782,SIZE, SIZE,3))
predicted[3564:5346] = (np.clip(autoencoder_lab.predict(test_gray_image[3564:5346].reshape(1782,SIZE, SIZE,1)),0.0,1.0).reshape(1782,SIZE, SIZE,3))
predicted[5346:7129] = (np.clip(autoencoder_lab.predict(test_gray_image[5346:7129].reshape(1783,SIZE, SIZE,1)),0.0,1.0).reshape(1783,SIZE, SIZE,3))
with tqdm(total=7129) as pbar: 
    for i in range(5100, 7129):
        plot_images(test_color_image[i],test_gray_image[i],predicted[i],i)
        pbar.update(1)

56/56 [==============================] - 6s 105ms/step


 28%|██▊       | 2029/7129 [07:25<18:38,  4.56it/s]
